In [1]:
import xarray
import numpy as np
import pandas as pd
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
# ### States
path_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/states.nc'
luh2_states = xarray.open_dataset(path_LUH2, engine="netcdf4", decode_times=False)
luh2_states_worldwide = luh2_states.isel(time=slice(1000, 1173))
luh2_states_worldwide['time'] = pd.date_range(start="1850-01-01", end="2022-01-01", freq='YS')

luh2_added_states_worldwide = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-IMAGE-ssp119-2-1-f_gn_2015-2100.nc",\
                                        engine="netcdf4", decode_times=False)
luh2_added_trans = luh2_added_states_worldwide.drop_vars(['lat_bounds', 'lon_bounds', 'time_bnds'])
luh2_added_trans.coords['time'] = pd.date_range(start='2015-01-01', end='2100-01-01', freq='YS')

# ### Static
static = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/staticData_quarterdeg.nc'
luh2_static = xarray.open_dataset(static, engine="netcdf4")

country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code.xlsx", engine="openpyxl")
ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['name'])

ccode_convert = np.zeros((720, 1440), dtype="<U64")

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

ccode_worldwide_int = luh2_static['ccode'].to_numpy().astype('int64')

for i in range(720):
    for j in range(1440):
        if ccode_worldwide_int[i][j] in ccode_dict.keys():
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "Unknown"

c:\Users\HEFRY ANESTI\AppData\Local\Programs\Python\Python39\lib\site-packages\xarray\coding\times.py:167: SerializationWarning: Ambiguous reference date string: 850-01-01 0:0:0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-01-01 0:0:0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [3]:
crop_class = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Crop Classification_latest.xlsx", engine="openpyxl", skiprows=1)
crop_class = crop_class.drop('Unnamed: 0', axis=1)
crop_class.rename(columns={'FAO Crops': 'Item'}, inplace=True)

fao_stat = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/fao_all_data.xlsx", engine="openpyxl")
fao_stat.dropna(axis=0, inplace=True)

fao_stat_yield = fao_stat[fao_stat['Element']=='Yield']
fao_stat_yield.insert(9, 'True Value', fao_stat_yield['Value']*100, True)
faostat_yield = fao_stat_yield.merge(crop_class, on='Item')

In [ ]:
grapc = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/GFRAC_GRAPC_Interpolation/GRAPC-RESAMPLE-1970-2100.nc", engine='netcdf4')
grapc_nfbfc = grapc.coords['NFBFC'].data.tolist()

grapc_nfbfc_ir_rf = []
for grapc_cls in grapc_nfbfc:
    if grapc_cls[0:2] == 'RF' or grapc_cls[0:2] == 'IR':
        grapc_nfbfc_ir_rf.append(grapc_cls)

In [ ]:
# fungsi interpolasi
def grid_level(kolom_sebelumnya, percent):
    return np.divide(np.multiply(kolom_sebelumnya, percent), 100)

def percent_change(grapc, kolom_sekarang, kolom_sebelumnya):
    if kolom_sebelumnya != 0:
        result = np.multiply(np.nan_to_num(np.divide(kolom_sekarang, kolom_sebelumnya)), 100)
    else:
        result = np.nan
    return grid_level(grapc, result)

In [ ]:
def grapc_interp(grapc_class, grapc_cls_idx, country):

    arr = np.zeros((56, 32, 720, 1440), dtype="float64")

    arr[0][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=0).sel(NGFBFC=grapc_class).to_numpy(), 0)
    arr[5][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=1).sel(NGFBFC=grapc_class).to_numpy(), 0)
    arr[10][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=2).sel(NGFBFC=grapc_class).to_numpy(), 0)
    arr[15][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=3).sel(NGFBFC=grapc_class).to_numpy(), 0)
    arr[20][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=4).sel(NGFBFC=grapc_class).to_numpy(), 0)
    arr[25][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=5).sel(NGFBFC=grapc_class).to_numpy(), 0)
    arr[30][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=6).sel(NGFBFC=grapc_class).to_numpy(), 0)
    arr[35][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=7).sel(NGFBFC=grapc_class).to_numpy(), 0)
    arr[40][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=8).sel(NGFBFC=grapc_class).to_numpy(), 0)
    arr[45][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=9).sel(NGFBFC=grapc_class).to_numpy(), 0)
    arr[50][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=10).sel(NGFBFC=grapc_class).to_numpy(), 0)
    arr[55][grapc_cls_idx] = np.where(country in ccode_convert, grapc['GRAPC_res'].isel(time=11).sel(NGFBFC=grapc_class).to_numpy(), 0)
    
    years = list(np.linspace(1970, 2025, 56, dtype="int64"))

    for column_idx, column in enumerate(years[1:]):
        if grapc_class[0:2] == 'RF':
            current_year_fao = (faostat_yield[(faostat_yield['Year'] == column)&(faostat_yield['Area'] == country)&
                                    (faostat_yield['IMAGE Classification']==grapc_class.replace('RF ',''))]['True Value'].sum())
            
            previous_year_fao = (faostat_yield[(faostat_yield['Year'] == column - 1)&(faostat_yield['Area'] == country)&
                                            (faostat_yield['IMAGE Classification'] == grapc_class.replace('RF ',''))]['True Value'].sum())

            current_year_grapc = arr[column_idx + 1]
            previous_year_grapc = arr[column_idx]
            
            if current_year_grapc.any() == 0.0: 
                arr[column_idx + 1] = percent_change(previous_year_grapc, current_year_fao, previous_year_fao)
        
        elif grapc_class[0:2] == 'IR':
            current_year_fao = (faostat_yield[(faostat_yield['Year'] == column)&(faostat_yield['Area'] == country)&
                                    (faostat_yield['IMAGE Classification']==grapc_class.replace('IR ',''))]['True Value'].sum())
            
            previous_year_fao = (faostat_yield[(faostat_yield['Year'] == column - 1)&(faostat_yield['Area'] == country)&
                                            (faostat_yield['IMAGE Classification'] == grapc_class.replace('IR ',''))]['True Value'].sum())
            
            current_year_grapc = arr[column_idx + 1]
            previous_year_grapc = arr[column_idx]
            
            if current_year_grapc.any() == 0.0: 
                arr[column_idx + 1] = percent_change(previous_year_grapc, current_year_fao, previous_year_fao)
    
    return arr

In [ ]:
worldwide_gfrac_interp = []
for cty_idx, country in enumerate(list(np.unique(ccode_convert))):
    worldwide_gfrac_interp.append([])
    for grapc_class_idx, grapc_class in enumerate(grapc_nfbfc_ir_rf):
        crops_interp_all_expand = np.expand_dims(grapc_interp(grapc_class, grapc_class_idx, country), axis=-1)
        worldwide_gfrac_interp[cty_idx].append(crops_interp_all_expand)

worldwide_gfrac_interp_arr = np.concatenate(worldwide_gfrac_interp, axis=-1)

In [ ]:
grapc_interp_netcdf = xarray.Dataset({
        "grapc_interp":(["country", "time", "lat", "lon", "nfbfc"], worldwide_grapc_interp_arr)
    },
    coords={
        "NFBFC": np.array(grapc_nfbfc_ir_rf, dtype='<U35'),
        "country": list(np.unique(ccode_convert)),
        "time":pd.date_range("1970-01-01", "2022-01-01", freq='YS'),
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy()
    })

grapc_interp_netcdf.to_netcdf("c:/Users/HEFRY ANESTI/Downloads/GRAPC-INTERPOLATION-WORLDWIDE-1970_2022.nc", mode='w', format="NETCDF4")